Custom evaluation

In [ ]:
import torch
import torchvision
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import numpy as np
import os, sys, json, random
import cv2 as cv
from matplotlib import pyplot as plt
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.engine import DefaultTrainer
from detectron2.utils.visualizer import ColorMode
from PARAMETERS import *


from tidecv import TIDE, datasets
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
import json
import statistics
import glob
import os
import cv2 as cv

# for evaluation without training
from PARAMETERS import *
from tkinter_dialog_custom import askdirectory

while True:
    CURRENT_DATASET = askdirectory(title="Select dataset folder", initialdir=PATH_TRAINING_DATA_SEGMENTATION)
    if os.path.exists(CURRENT_DATASET): break



In [ ]:
cfg = get_cfg()
network_config = r"COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"
cfg.merge_from_file(model_zoo.get_config_file(network_config))
cfg.OUTPUT_DIR =  askdirectory(initialdir=PATH_MODELS_SEGMENTATION,title="Select trained model")
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.DATASETS.TEST = ("TCM_test", )
cfg.DATALOADER.NUM_WORKERS = 1
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4
cfg.OUTPUT_DIR =  PATH_TRAINING_OUTPUT_DIR_SEGMENTATION


In [ ]:
def custom_eval_old(segmentation_predictor,path_current_dataset):
    pth = os.path.join(path_current_dataset, r'test').replace('\\\\','\\').replace('\\','/')
    failures_dictionary = DICTIONARY_FAILURES
    
    F1_score_global = []
    Specificity_global = []
    Accuracy_global = []
    
    F1_score_global_stepienie, F1_score_global_narost, F1_score_global_zatarcie, F1_score_global_wykruszenie = [],[],[],[]

    for label in glob.glob(f'{pth}/*json'): # iterate through eval dataset
        base_name = (label.replace('\\\\','\\').replace('\\','/').split('/')[-1]).split('.')[0]
        file = open(label, 'r')
        json_label = json.load(file )
        im = cv.imread(os.path.join(pth,f"{base_name}.png"))
        print(os.path.join(pth,f"{base_name}.png"))
        
        label_outputs = {
            "wykruszenie": np.zeros_like(im),
            "narost": np.zeros_like(im),
            "stepienie": np.zeros_like(im),
            "zatarcie": np.zeros_like(im)
        }

        # Iterate over labels from .json file, merge it into 4 categories, draw it as bitmaps
        for shape in json_label["shapes"]:
            point_list = np.array(shape['points'])
            if len(point_list) > 2:
                pts = point_list.reshape((-1, 1, 2))  
                for class_name in failures_dictionary.values(): # Iterate over all instances classes
                    if(shape["label"]==class_name):
                        label_outputs[class_name] = cv.fillPoly(label_outputs[class_name], np.int32([pts]), (255,255,255))
                        
        # Segmentation model inference
        predictions = segmentation_predictor(im) # Make prediction 
        pred_masks = predictions["instances"].to("cpu").pred_masks.numpy()
        pred_classes = predictions["instances"].to("cpu").pred_classes.numpy()
        num_instances = pred_masks.shape[0] 
        pred_masks = np.moveaxis(pred_masks, 0, -1)
        pred_masks_instance = []
        output = np.zeros_like(im)
        
        # Contains merged bitmaps for particular failures classes
        inference_outputs = {
            "wykruszenie": np.zeros_like(im),
            "narost": np.zeros_like(im),
            "stepienie": np.zeros_like(im),
            "zatarcie": np.zeros_like(im)
        }
        # Contains temporary data used during merging
        pred_masks_instance = {   
            "wykruszenie": [],
            "narost": [],
            "stepienie": [],
            "zatarcie": []
        }

        # Iterate over predicted defects, search for duplicated instances of the same class and merge them into single bitmap.
        for i in range(num_instances): 
            for class_id in failures_dictionary: # Iterate over all instances classes
                if(pred_classes[i] == class_id): 
                    failure_class = failures_dictionary[class_id] # Get name of the current class
                    pred_masks_instance[failure_class].append(pred_masks[:, :, i:(i+1)]) 
                    inference_outputs[failure_class] = np.where(pred_masks_instance[failure_class][-1] == True, 255, inference_outputs[failure_class])
    
        
        F1_score_mean = 0 
        F1_counter = 0
        F1_score_stepienie_m, F1_score_wykruszenie_m, F1_score_zatarcie_m, F1_score_narost_m = 0,0,0,0

        for class_name in failures_dictionary.values():
        
            pred = cv.cvtColor(inference_outputs[class_name], cv.COLOR_BGR2GRAY)
            label = cv.cvtColor(label_outputs[class_name], cv.COLOR_BGR2GRAY)
            bitwiseOr = cv.bitwise_or(pred, label)
            bitwiseAnd = cv.bitwise_and(pred, label) 
            bitwiseXor = cv.bitwise_xor(pred, label)
            TP = bitwiseAnd  # TP - a sample is predicted to be positive and its label is actually positive
            TN = cv.bitwise_not(bitwiseOr) # TN - a sample is predicted to be negative and its label is actually negative
            FP = cv.bitwise_xor(TP,pred) # FP - a sample is predicted to be positive and its label is actually negative
            FN = cv.bitwise_and(cv.bitwise_not(pred),cv.bitwise_xor(label,TP)) # FN - a sample is predicted to be negative and its label is actually positive

            TP = cv.countNonZero(TP)
            TN = cv.countNonZero(TN)
            FP = cv.countNonZero(FP)
            FN = cv.countNonZero(FN)
            #acc = (TP+TN)/(TP+TN+FP+FN)
            precision = TP/(TP+FP) if (TP+FP) != 0 else 0
            recall = TP/(TP+FN) if (TP+FN) != 0 else 0
            F1_score = (2*precision*recall)/(precision+recall) if (precision+recall) != 0 else -1

            if F1_score != -1:
                if class_name == "stepienie": F1_score_stepienie_m  += F1_score
                if class_name == "wykruszenie": F1_score_wykruszenie_m  += F1_score
                if class_name == "narost": F1_score_narost_m  += F1_score
                if class_name == "zatarcie": F1_score_zatarcie_m  += F1_score

        if 2 in pred_classes: F1_score_global_stepienie.append(F1_score_stepienie_m) 
        if 1 in pred_classes: F1_score_global_narost.append(F1_score_narost_m)
        if 3 in pred_classes: F1_score_global_zatarcie.append(F1_score_zatarcie_m) 
        if 0 in pred_classes: F1_score_global_wykruszenie.append(F1_score_wykruszenie_m)


    F1_score_global = F1_score_global_wykruszenie + F1_score_global_zatarcie + F1_score_global_narost + F1_score_global_stepienie

    if len(F1_score_global)==0: F1_score_global.append(-1)
    if len(F1_score_global_wykruszenie)==0: F1_score_global_wykruszenie.append(-1)
    if len(F1_score_global_zatarcie)==0: F1_score_global_zatarcie.append(-1)
    if len(F1_score_global_narost)==0: F1_score_global_narost.append(-1)
    if len(F1_score_global_stepienie)==0: F1_score_global_stepienie.append(-1)

    print(statistics.mean(F1_score_global), statistics.mean(F1_score_global_stepienie), statistics.mean(F1_score_global_narost), statistics.mean(F1_score_global_zatarcie), statistics.mean(F1_score_global_wykruszenie))
    return statistics.mean(F1_score_global), statistics.mean(F1_score_global_stepienie), statistics.mean(F1_score_global_narost), statistics.mean(F1_score_global_zatarcie), statistics.mean(F1_score_global_wykruszenie)

        

In [ ]:
def custom_eval(segmentation_predictor,path_current_dataset):
    pth = os.path.join(path_current_dataset, r'test').replace('\\\\','\\').replace('\\','/')

    metrics = dict()
    results = dict()
    
    for value in list(DICTIONARY_FAILURES.values())+["Global"]:
            results[value] = {
                "Recall":0,
                "Precision":0,
                "Accuracy":0,
                "Specificity":0,
                "F1":0,
            }
    
    for value in list(DICTIONARY_FAILURES.values())+["Global"]:
            metrics[value] = {
                "TP":[],
                "TN":[],
                "FP":[],
                "FN":[],
            }
    

    for label in glob.glob(f'{pth}/*json'): # iterate through eval dataset
        base_name = (label.replace('\\\\','\\').replace('\\','/').split('/')[-1]).split('.')[0]
        file = open(label, 'r')
        json_label = json.load(file )
        im = cv.imread(os.path.join(pth,f"{base_name}.png"))
        pred_classes, image_metrics = calculate_metrics_for_image(im, json_label, segmentation_predictor)
        
        for class_name in list(image_metrics.keys()):
            for metric_name, metric_val in zip(list(image_metrics[class_name].keys()),list(image_metrics[class_name].values())):
                metrics[class_name][metric_name].append(metric_val)
                metrics["Global"][metric_name].append(metric_val)

    # calculate metrics for entire run
    for class_name in list(results.keys()):
        TP = sum(metrics[class_name]["TP"])
        TN = sum(metrics[class_name]["TN"])
        FP = sum(metrics[class_name]["FP"])
        FN = sum(metrics[class_name]["FN"])
        accuracy = (TP+TN)/(TP+TN+FP+FN) # Accuracy is a measure of how many of the predictions were correct
        precision = TP/(TP+FP) if (TP+FP) != 0 else -1 # Precision is a measure of how many of the positive predictions made are correct (true positives)
        recall = TP/(TP+FN) if (TP+FN) != 0 else -1 # Recall is a measure of how many of the positive cases the classifier correctly predicted
        specificity = TN / (TN + FP) # Specificity is a measure of how many negative predictions made are correct (true negatives). 
        F1_score = (2*precision*recall) / (precision+recall) if (precision != -1 and recall != -1) else -1
        results[class_name]["Recall"] = recall
        results[class_name]["Accuracy"] = accuracy
        results[class_name]["Precision"] = precision
        results[class_name]["Specificity"] = specificity
        results[class_name]["F1"] = F1_score

   
    for class_name in list(results.keys()):
        print(
            f"""
            For class {class_name}
                F1 = {results[class_name]["F1"]}
            """
        )
    return results

def calculate_metrics_for_image(im, json_label, segmentation_predictor):
        #cv.imshow("test",im)
        im_h, im_w, _ = im.shape

        label_outputs = dict()
        for value in DICTIONARY_FAILURES.values():
            label_outputs[str(value)] = np.zeros_like(im)
        
        # Iterate over labels from .json file, merge it into 4 categories, draw it as bitmaps
        for shape in json_label["shapes"]:
            point_list = np.array(shape['points'])
            if len(point_list) > 2:
                pts = point_list.reshape((-1, 1, 2))  
                for class_name in DICTIONARY_FAILURES.values(): # Iterate over all instances classes
                    if(shape["label"]==class_name):
                        label_outputs[class_name] = cv.fillPoly(label_outputs[class_name], np.int32([pts]), (255,255,255))
                        
        # Segmentation model inference
        predictions = segmentation_predictor(im) # Make prediction 
        pred_masks = predictions["instances"].to("cpu").pred_masks.numpy()
        pred_classes = predictions["instances"].to("cpu").pred_classes.numpy()
        num_instances = pred_masks.shape[0] 
        pred_masks = np.moveaxis(pred_masks, 0, -1)
        
        # Contains merged bitmaps for particular failures classes
        inference_outputs = dict()
        for value in DICTIONARY_FAILURES.values():
            inference_outputs[str(value)] = np.zeros_like(im)

        # Contains temporary data used during merging
        pred_masks_instance = dict()
        for value in DICTIONARY_FAILURES.values():
            pred_masks_instance[str(value)] = []

        # Iterate over predicted defects, search for duplicated instances of the same class and merge them into single bitmap.
        for i in range(num_instances): 
            for class_id in DICTIONARY_FAILURES: # Iterate over all instances classes
                if(pred_classes[i] == class_id): 
                    failure_class = DICTIONARY_FAILURES[class_id] # Get name of the current class
                    pred_masks_instance[failure_class].append(pred_masks[:, :, i:(i+1)]) 
                    inference_outputs[failure_class] = np.where(pred_masks_instance[failure_class][-1] == True, 255, inference_outputs[failure_class])
    
        image_metrics = dict()

        for value in DICTIONARY_FAILURES.values():
            image_metrics[str(value)] = {
                "TP":0.0,
                "TN":0.0,
                "FP":0.0,
                "FN":0.0,
            }

        for class_name in DICTIONARY_FAILURES.values():
            # iterate through all potential failures
        
            pred = cv.cvtColor(inference_outputs[class_name], cv.COLOR_BGR2GRAY)
            label = cv.cvtColor(label_outputs[class_name], cv.COLOR_BGR2GRAY)
            bitwiseOr = cv.bitwise_or(pred, label)
            bitwiseAnd = cv.bitwise_and(pred, label) 
            #bitwiseXor = cv.bitwise_xor(pred, label)

            TP = bitwiseAnd  # TP - a sample is predicted to be positive and its label is actually positive
            TN = cv.bitwise_not(bitwiseOr) # TN - a sample is predicted to be negative and its label is actually negative
            FP = cv.bitwise_xor(TP,pred) # FP - a sample is predicted to be positive and its label is actually negative
            FN = cv.bitwise_and(cv.bitwise_not(pred),cv.bitwise_xor(label,TP)) # FN - a sample is predicted to be negative and its label is actually positive

            im_size = im_h*im_w

            TP = cv.countNonZero(TP) / im_size # normalize to image size
            TN = cv.countNonZero(TN) / im_size
            FP = cv.countNonZero(FP) / im_size
            FN = cv.countNonZero(FN) / im_size

            image_metrics[class_name]["TP"] = cv.countNonZero(TP)
            image_metrics[class_name]["TN"] = cv.countNonZero(TN)
            image_metrics[class_name]["FP"] = cv.countNonZero(FP)
            image_metrics[class_name]["FN"] = cv.countNonZero(FN)

        return pred_classes, image_metrics


In [ ]:
F1_scores = []
F1_scores_stepienie, F1_scores_wykruszenie, F1_scores_zatarcie, F1_scores_narost = [],[],[],[]
Accuracy, Specificity = [],[]
tresholds = []

output_metrics = dict()
for value in list(DICTIONARY_FAILURES.values())+["Global"]:
    output_metrics[value] = {
            "Recall":[],
            "Precision":[],
            "Accuracy":[],
            "Specificity":[],
            "F1":[],
        }

F1_scores_global_old = list()
F1_scores_stepienie_old = list()
F1_scores_narost_old =  list()
F1_scores_zatarcie_old =  list()
F1_scores_wykruszenie_old =  list()

for TRESHOLD in np.arange(0.3, 1.0, 0.05): #Iterate over various tresholds to find best results
    TRESHOLD = float(round(TRESHOLD,2))
    tresholds.append(TRESHOLD*100)
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = TRESHOLD
    predictor = DefaultPredictor(cfg)
    test_metadata = MetadataCatalog.get("TCM_test")
    print("Treshold",TRESHOLD)
    # NEW EVAL
    eval_results = custom_eval(predictor, CURRENT_DATASET)
    # OLD EVAL
    eval_results_old = custom_eval_old(predictor, CURRENT_DATASET)
    F1_scores_global_old.append(eval_results_old[0])
    F1_scores_stepienie_old.append(eval_results_old[1])
    F1_scores_narost_old.append(eval_results_old[2])
    F1_scores_zatarcie_old.append(eval_results_old[3])
    F1_scores_wykruszenie_old.append(eval_results_old[4])

    for class_name in list(eval_results.keys()):
        for metric in list(eval_results[class_name].keys()):
            output_metrics[class_name][metric].append(eval_results[class_name][metric])

In [ ]:
experiment_folder = PATH_TRAINING_OUTPUT_DIR_SEGMENTATION

fig_class_scores, axes = plt.subplots(nrows=2,ncols=2)
axes = axes.flatten()

fig_class_scores.suptitle("Metrics (Classes)")
class_titles = list(DICTIONARY_FAILURES.values())
for i,ax in enumerate(axes):
    current_class_data = output_metrics[class_titles[i]]
    ax.set_xlabel('Treshold [%]')
    ax.set_ylabel('Metric')
    ax.set_title(class_titles[i])
    ax.grid(alpha=0.5)
    for key,data in zip(list(current_class_data.keys()),list(current_class_data.values())):
        ax.plot(tresholds,data,marker="o",linestyle="none",label=key)
    ax.legend()
fig_class_scores.set_size_inches(10,10)
fig_class_scores.tight_layout()
fig_class_scores.savefig(os.path.join(experiment_folder,'F1_class_scores.png'))
fig_class_scores.show()

fig_global_metrics, axes = plt.subplots(ncols=5)
axes = axes.flatten()

fig_global_metrics.suptitle("Metrics (Global)")
for i,ax in enumerate(axes):
    metric_key,metric_val = list(output_metrics["Global"].keys())[i], list(output_metrics["Global"].values())[i]
    ax.plot(tresholds,metric_val,marker="o",linestyle="none",label=metric_key)
    ax.set_xlabel('Treshold [%]')
    ax.set_ylabel('Metric')
    ax.grid(alpha=0.5)
    ax.set_title(metric_key)
fig_global_metrics.set_size_inches(16,6)
fig_global_metrics.tight_layout()
fig_global_metrics.savefig(os.path.join(experiment_folder,'Global_metrics.png'))
fig_global_metrics.show()

In [ ]:
global_F1_max_old = max(F1_scores_global_old)

params = {
    "F1": global_F1_max_old,
    "tresh": tresholds[F1_scores_global_old.index(global_F1_max_old)],
    "F1_stepienie": F1_scores_stepienie_old[F1_scores_global_old.index(global_F1_max_old)],
    "F1_narost": F1_scores_narost_old[F1_scores_global_old.index(global_F1_max_old)],
    "F1_zatarcie": F1_scores_zatarcie_old[F1_scores_global_old.index(global_F1_max_old)],
    "F1_wykruszenie": F1_scores_wykruszenie_old[F1_scores_global_old.index(global_F1_max_old)],
}

print("OLD PARAMETERS")
print(params)